<a href="https://colab.research.google.com/github/ibshafique/mlops_with_poridhi/blob/main/prerequisite_projects/WQU_DS/01_housing_in_mexico/mexico_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

## Importing Relevant Libraries

In [ ]:
import pandas as pd

## Importing Datasets

In [ ]:
url_1 = "https://raw.githubusercontent.com/ibshafique/mlops_with_poridhi/refs/heads/main/prerequisite_projects/WQU_DS/01_housing_in_mexico/dataset/mexico-real-estate-1.csv"
url_2 = "https://raw.githubusercontent.com/ibshafique/mlops_with_poridhi/refs/heads/main/prerequisite_projects/WQU_DS/01_housing_in_mexico/dataset/mexico-real-estate-2.csv"
url_3 = "https://raw.githubusercontent.com/ibshafique/mlops_with_poridhi/refs/heads/main/prerequisite_projects/WQU_DS/01_housing_in_mexico/dataset/mexico-real-estate-3.csv"

In [ ]:
df1 = pd.read_csv(url_1)
df2 = pd.read_csv(url_2)
df3 = pd.read_csv(url_3)

# Data Insights

## Dataset 1

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.info()

## Dataset 2

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.info()

## Dataset 3

In [ ]:
df3.head()

In [ ]:
df3.shape

In [ ]:
df3.info()